# 使用斷詞套件，Jieba

In [1]:
import jieba
jieba.set_dictionary('dict.txt.big')   #載入繁體辭典

# Jieba 有四種斷詞模式:
1. 精確模式:     基本款

2. 全模式:       將cut_all設為true，會將可以斷的詞再繼續斷開，列出所有可能的詞

3. paddle模式:   Jieba導入一個深度學習框架PaddlePaddle來進行分詞，Paddle模式相比传统的基于规则的分词算法，在一些特定的场
                景下可能会有更好的效果。據說是利用GRU(循環神經網路,Gate Recurrent Unit) ，必須要先利用 
                jieba.enable_paddle()來啟動。

4. 搜尋引擎模式: 這個模式跟全模式很像，都可以將詞再更切分


精確模式

In [11]:
text_after_jieba = jieba.cut("我愛自然語言處理", cut_all=False) # cut_all=False 為精確模式
print("精確模式: " + "/ ".join(text_after_jieba))

精確模式: 我/ 愛/ 自然語言/ 處理


In [9]:
text_after_jieba  # 在使用jieba分詞後打印text_after_jieba沒有任何內容，可能是因為jieba.cut返回的是一個生成器（generator）
                  #，而不是一個列表或字符串。生成器是一種特殊的對象，它需要進行迭代才能獲取其中的元素。

<generator object Tokenizer.cut at 0x000001900E05F970>

In [13]:
text_after_jieba = jieba.cut("我愛自然語言處理", cut_all=False)
for i in text_after_jieba:
    print(i)

我
愛
自然語言
處理


全模式

In [5]:
text_after_jieba = jieba.cut("我愛自然語言處理", cut_all=True) # 全模式
print("全模式: " + "/ ".join(text_after_jieba))

全模式: 我/ 愛/ 自然/ 自然語言/ 語言/ 處理


paddle模式

In [20]:
import paddle

paddle.enable_static()  # 進入靜態圖模式
jieba.enable_paddle()
seg_list = jieba.cut("我愛自然語言處理", use_paddle=True)  # 使用Paddle模式
print("paddle: " + "/ ".join(seg_list))

Paddle enabled successfully......


paddle: 我/ 愛/ 自然/ 語言/ 處理


搜尋引擎模式

In [21]:
text_after_jieba = jieba.cut_for_search('我愛自然語言處理')
print("搜尋引擎: " + "/".join(text_after_jieba))

搜尋引擎: 我/愛/自然/語言/自然語言/處理


# 辨識新字詞

若Jieba遇到新詞，可以透過HMM( Hidden Markov Models）來預測 找出『未登錄詞』

HMM 模型就是一種統計斷詞的方法，未登錄詞就是想成詞典中沒有這種詞，必須經過統計來得知新的詞語。

In [29]:
words = jieba.cut("我今天想吃東西", HMM=True)
words_list = []
for word in words:
    words_list.append(word)

print(f"HMM output:{words_list}")

HMM output:['我', '今天', '想', '吃', '東西']


# 載入自定義詞典

In [32]:
text = 'NBA金洲勇士隊2023總冠軍'
words = jieba.cut(text)
words_list = []
for word in words:
    words_list.append(word)

print(f'output:{words_list}')

jieba.load_userdict(r"C:\Users\yifun\Desktop\python\define.txt.txt")   #載入我自己定義的詞典

words = jieba.cut(text)
words_list = []
for word in words:
    words_list.append(word)

print(f'output:{words_list}')

output:['NBA', '金洲', '勇士隊', '2023', '總冠軍']
output:['NBA', '金洲勇士', '隊', '2023', '總冠軍']


# pos tagging( 詞性標註 , Part-of-speech tagging) 

In [35]:
import jieba.posseg as pseg

words = jieba.posseg.cut('我愛自然語言處理')
for word, flag in words:
    print(f'{word} {flag}')

我 r
愛 v
自然語言 l
處理 v


In [42]:
import nltk
from nltk.tokenize import word_tokenize
text = word_tokenize("Hello welcome to the world of to learn Categorizing and POS Tagging with NLTK and Python")
nltk.pos_tag(text)

[('Hello', 'NNP'),
 ('welcome', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('world', 'NN'),
 ('of', 'IN'),
 ('to', 'TO'),
 ('learn', 'VB'),
 ('Categorizing', 'NNP'),
 ('and', 'CC'),
 ('POS', 'NNP'),
 ('Tagging', 'NNP'),
 ('with', 'IN'),
 ('NLTK', 'NNP'),
 ('and', 'CC'),
 ('Python', 'NNP')]

## 常見的方法:

- Rule-Based：自訂 rules 來標記單詞，如看到 ed、i，或是看到'看'、'打'就標注 verb。
- Probabilistic：使用條件機率的原理，預測單詞詞性，常見如 CRF、HMM，此方法也是深度學習出來前，最常見且效果最好的標注方式。
- Deep Learning：使用深度學習模型預測標註詞性，例如使用LSTM, BERT等方法進行多對多的訓練，這裡有很多相關的資訊可以參考

# 文本/詞表示方式
- BOW(Bag of word)詞袋
- one-hot represtation
- tf-idf  ('Term Frequency(詞頻)'與 'Inverted Document Frequency(逆詞頻)')

In [48]:
import jieba
import math

# 載入繁體
jieba.set_dictionary('dict.txt.big')

text_a = '從GPT-3衍生改良而來的Codex模型，能夠將使用者的自然語言指令轉換為程式碼，OpenAI現在以私人測試的方式釋出CodexAPI'
text_b = 'Blender2.0除了能即時搜尋網路資訊，臉書也為其打造新的神經模組，可根據之前使用者與它的聊天脈絡來累積記憶'

開始斷詞與紀錄每個詞出現的次數

In [63]:
#斷詞
seg_1 =jieba.lcut(text_a)
seg_2 = jieba.lcut(text_b)
unique_words = set(seg_1).union(set(seg_2))   #取聯集，取得所有文件中的單詞
#建立兩個新字典，分別存兩篇文章詞的出現次數
num_words_1 = dict.fromkeys(unique_words,0)     #用來記錄第一個文章在全部的詞中，有哪些出現，並累積次數，從0開始
num_words_2 = dict.fromkeys(unique_words,0)     #   記錄第二篇文章 ~

for i in seg_1:
    num_words_1[i] += 1

for j in seg_2:
    num_words_2[j] += 1

print(num_words_1)
print(num_words_2)

{'模型': 1, '為': 1, '能夠': 1, '資訊': 0, '記憶': 0, '可': 0, '方式': 1, '而來': 1, '除了': 0, 'GPT': 1, '指令': 1, '也': 0, '打造': 0, '以': 1, '程式碼': 1, '現在': 1, 'CodexAPI': 1, '搜尋': 0, 'Codex': 1, '模組': 0, '根據': 0, '累積': 0, '新': 0, '私人': 1, '，': 2, '與': 0, '改良': 1, '從': 1, '它': 0, '即時': 0, '-': 1, '來': 0, '3': 1, 'Blender2.0': 0, '衍生': 1, '臉書': 0, '釋出': 1, '能': 0, '轉換': 1, '將': 1, '脈絡': 0, '自然語言': 1, 'OpenAI': 1, '聊天': 0, '測試': 1, '之前': 0, '神經': 0, '其': 0, '網路': 0, '使用者': 1, '的': 3}
{'模型': 0, '為': 1, '能夠': 0, '資訊': 1, '記憶': 1, '可': 1, '方式': 0, '而來': 0, '除了': 1, 'GPT': 0, '指令': 0, '也': 1, '打造': 1, '以': 0, '程式碼': 0, '現在': 0, 'CodexAPI': 0, '搜尋': 1, 'Codex': 0, '模組': 1, '根據': 1, '累積': 1, '新': 1, '私人': 0, '，': 2, '與': 1, '改良': 0, '從': 0, '它': 1, '即時': 1, '-': 0, '來': 1, '3': 0, 'Blender2.0': 1, '衍生': 0, '臉書': 1, '釋出': 0, '能': 1, '轉換': 0, '將': 0, '脈絡': 1, '自然語言': 0, 'OpenAI': 0, '聊天': 1, '測試': 0, '之前': 1, '神經': 1, '其': 1, '網路': 1, '使用者': 1, '的': 2}


# 實作TF與IDF的function

- 詞頻（TF）=某個詞在文檔中出現的次數/文檔的總詞數

- 逆文檔頻率（IDF）=log(語料庫的文檔總數/(包含該詞的文檔數+1))

In [65]:
def get_TF_value(w_dict, text_seg_len):
    tf_dict = {}
    
    for w, count in w_dict.items():
        # 計算tf的公式
        tf_dict[w] = count / float(text_seg_len)
    
    return tf_dict

In [66]:
def get_IDF_value(text_list, all_words):
    
    idf_dict = dict.fromkeys(all_words.keys(), 0)
    
    for text in text_list:
        for w, val in text.items():
            # 表示出現過在一次文本中         
            if val > 0:
                idf_dict[w] += 1
    
    for w, val in idf_dict.items():
        # 計算idf的公式
        idf_dict[w] = math.log(len(text_list) / float(val))
    return idf_dict

In [70]:
tf_1 = get_TF_value(num_words_1, len(seg_1))
tf_2 = get_TF_value(num_words_2, len(seg_2))

idf = get_IDF_value([num_words_1, num_words_2], num_words_1)

# 計算tfidf
tfidf_1 = {}
tfidf_2 = {}
for w, val in tf_1.items():
    tfidf_1[w] = val * idf[w]

for w, val in tf_2.items():
    tfidf_2[w] = val * idf[w]

In [71]:
tfidf_1

{'模型': 0.023104906018664842,
 '為': 0.0,
 '能夠': 0.023104906018664842,
 '資訊': 0.0,
 '記憶': 0.0,
 '可': 0.0,
 '方式': 0.023104906018664842,
 '而來': 0.023104906018664842,
 '除了': 0.0,
 'GPT': 0.023104906018664842,
 '指令': 0.023104906018664842,
 '也': 0.0,
 '打造': 0.0,
 '以': 0.023104906018664842,
 '程式碼': 0.023104906018664842,
 '現在': 0.023104906018664842,
 'CodexAPI': 0.023104906018664842,
 '搜尋': 0.0,
 'Codex': 0.023104906018664842,
 '模組': 0.0,
 '根據': 0.0,
 '累積': 0.0,
 '新': 0.0,
 '私人': 0.023104906018664842,
 '，': 0.0,
 '與': 0.0,
 '改良': 0.023104906018664842,
 '從': 0.023104906018664842,
 '它': 0.0,
 '即時': 0.0,
 '-': 0.023104906018664842,
 '來': 0.0,
 '3': 0.023104906018664842,
 'Blender2.0': 0.0,
 '衍生': 0.023104906018664842,
 '臉書': 0.0,
 '釋出': 0.023104906018664842,
 '能': 0.0,
 '轉換': 0.023104906018664842,
 '將': 0.023104906018664842,
 '脈絡': 0.0,
 '自然語言': 0.023104906018664842,
 'OpenAI': 0.023104906018664842,
 '聊天': 0.0,
 '測試': 0.023104906018664842,
 '之前': 0.0,
 '神經': 0.0,
 '其': 0.0,
 '網路': 0.0,
 '使用者': 0.0

In [72]:
tfidf_2

{'模型': 0.0,
 '為': 0.0,
 '能夠': 0.0,
 '資訊': 0.023104906018664842,
 '記憶': 0.023104906018664842,
 '可': 0.023104906018664842,
 '方式': 0.0,
 '而來': 0.0,
 '除了': 0.023104906018664842,
 'GPT': 0.0,
 '指令': 0.0,
 '也': 0.023104906018664842,
 '打造': 0.023104906018664842,
 '以': 0.0,
 '程式碼': 0.0,
 '現在': 0.0,
 'CodexAPI': 0.0,
 '搜尋': 0.023104906018664842,
 'Codex': 0.0,
 '模組': 0.023104906018664842,
 '根據': 0.023104906018664842,
 '累積': 0.023104906018664842,
 '新': 0.023104906018664842,
 '私人': 0.0,
 '，': 0.0,
 '與': 0.023104906018664842,
 '改良': 0.0,
 '從': 0.0,
 '它': 0.023104906018664842,
 '即時': 0.023104906018664842,
 '-': 0.0,
 '來': 0.023104906018664842,
 '3': 0.0,
 'Blender2.0': 0.023104906018664842,
 '衍生': 0.0,
 '臉書': 0.023104906018664842,
 '釋出': 0.0,
 '能': 0.023104906018664842,
 '轉換': 0.0,
 '將': 0.0,
 '脈絡': 0.023104906018664842,
 '自然語言': 0.0,
 'OpenAI': 0.0,
 '聊天': 0.023104906018664842,
 '測試': 0.0,
 '之前': 0.023104906018664842,
 '神經': 0.023104906018664842,
 '其': 0.023104906018664842,
 '網路': 0.02310490601866

# 用TF/IDF表示成句字/文本

In [73]:
# 創建一個表示text a的list
bow_a = []
# 將tfidf_a帶入即可
for w, val in tfidf_a.items():
    bow_a.append(val)

print(bow_a)

[0.023104906018664842, 0.0, 0.023104906018664842, 0.0, 0.0, 0.0, 0.023104906018664842, 0.023104906018664842, 0.0, 0.023104906018664842, 0.023104906018664842, 0.0, 0.0, 0.023104906018664842, 0.023104906018664842, 0.023104906018664842, 0.023104906018664842, 0.0, 0.023104906018664842, 0.0, 0.0, 0.0, 0.0, 0.023104906018664842, 0.0, 0.0, 0.023104906018664842, 0.023104906018664842, 0.0, 0.0, 0.023104906018664842, 0.0, 0.023104906018664842, 0.0, 0.023104906018664842, 0.0, 0.023104906018664842, 0.0, 0.023104906018664842, 0.023104906018664842, 0.0, 0.023104906018664842, 0.023104906018664842, 0.0, 0.023104906018664842, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# 使用CKIP套件

In [1]:
from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
)

# masked language model (ALBERT, BERT)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForMaskedLM.from_pretrained('ckiplab/albert-tiny-chinese') # or other models above

# casual language model (GPT2)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForCausalLM.from_pretrained('ckiplab/gpt2-base-chinese') # or other models above

# nlp task model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForTokenClassification.from_pretrained('ckiplab/albert-tiny-chinese-ws') # or other models above

C:\Users\yifun\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\yifun\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yifun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, s

In [4]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [5]:
# Initialize drivers
ws_driver  = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")
ner_driver = CkipNerChunker(model="bert-base")

In [8]:
# Use CPU
ws_driver = CkipWordSegmenter(device=-1)

# Use GPU:0
ws_driver = CkipWordSegmenter(device=-1)

In [9]:
# Input text
text = [
   "傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。",
   "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會，預料她將會很順利通過參議院支持，成為該國有史以來第一位的華裔女性內閣成員。",
   "空白 也是可以的～",
]

# Run pipeline
ws  = ws_driver(text)
pos = pos_driver(ws)
ner = ner_driver(text)

Inference: 100%|██████████| 1/1 [00:00<00:00,  2.26it/s]


In [10]:
# Pack word segmentation and part-of-speech results
def pack_ws_pos_sentece(sentence_ws, sentence_pos):
   assert len(sentence_ws) == len(sentence_pos)
   res = []
   for word_ws, word_pos in zip(sentence_ws, sentence_pos):
      res.append(f"{word_ws}({word_pos})")
   return "\u3000".join(res)

# Show results
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
   print(sentence)
   print(pack_ws_pos_sentece(sentence_ws, sentence_pos))
   for entity in sentence_ner:
      print(entity)
   print()

傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。
傅達仁(Nb)　今(Nd)　將(D)　執行(VC)　安樂死(Na)　，(COMMACATEGORY)　卻(D)　突然(D)　爆出(VJ)　自己(Nh)　20(Neu)　年(Nf)　前(Ng)　遭(P)　緯來(Nb)　體育台(Na)　封殺(VC)　，(COMMACATEGORY)　他(Nh)　不(D)　懂(VK)　自己(Nh)　哪裡(Ncd)　得罪到(VC)　電視台(Nc)　。(PERIODCATEGORY)
NerToken(word='傅達仁', ner='PERSON', idx=(0, 3))
NerToken(word='20年', ner='DATE', idx=(18, 21))
NerToken(word='緯來體育台', ner='ORG', idx=(23, 28))

美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會，預料她將會很順利通過參議院支持，成為該國有史以來第一位的華裔女性內閣成員。
美國(Nc)　參議院(Nc)　針對(P)　今天(Nd)　總統(Na)　布什(Nb)　所(D)　提名(VC)　的(DE)　勞工部長(Na)　趙小蘭(Nb)　展開(VC)　認可(VC)　聽證會(Na)　，(COMMACATEGORY)　預料(VE)　她(Nh)　將(D)　會(D)　很(Dfa)　順利(VH)　通過(VC)　參議院(Nc)　支持(Nv)　，(COMMACATEGORY)　成為(VG)　該(Nes)　國(Nc)　有史以來(D)　第一(Neu)　位(Nf)　的(DE)　華裔(Na)　女性(Na)　內閣(Na)　成員(Na)　。(PERIODCATEGORY)
NerToken(word='美國參議院', ner='ORG', idx=(0, 5))
NerToken(word='今天', ner='LOC', idx=(7, 9))
NerToken(word='布什', ner='PERSON', idx=(11, 13))
NerToken(word='勞工部長', ner='ORG', idx=(17, 21))
NerToken(word='趙小蘭', ner='PERSON', idx=(21, 24))
NerToken(word='認